# Notebook

In [235]:
import math
import cmath
import yaml

## Book Keeping

In [236]:
MODEL = "PE1R_112_M4"
VERBOSITY = 1

In [237]:
RATED_SPEED = 1500
RATED_FREQUENCY = 50
RATE_POWER = 4e3
VOLTAGE = 320
POWER_FACTOR = 0.99
RATED_CURRENT = 7.9
PHASE_RESISTANCE = 0.9
MAGNETISING_INDUCTANCE = 10.5e-3
LEAKAGE_INDUCTANCE = 2.5e-3
MOTOR_MOMENT_OF_INERTIA = 0.009

In [238]:
P = (120 * RATED_FREQUENCY) / RATED_SPEED
we = RATED_FREQUENCY * (2 * math.pi)
ws = RATED_SPEED * (math.pi / 30)
wr = ws

if VERBOSITY:
    print("P  :: {:.4f}".format(P))
    print("we  :: {:.4f}".format(we))
    print("ws  :: {:.4f}".format(ws))
    print("wr  :: {:.4f}".format(wr))

P  :: 4.0000
we  :: 314.1593
ws  :: 157.0796
wr  :: 157.0796


In [239]:
Ld = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE
Lq = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE
Ls = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE

if VERBOSITY:
    print("Ld :: {:.4f}m".format(Ld * 1000))
    print("Lq :: {:.4f}m".format(Lq * 1000))
    print("Ls :: {:.4f}m".format(Ls * 1000))

Ld :: 18.2500m
Lq :: 18.2500m
Ls :: 18.2500m


In [240]:
Va = VOLTAGE / math.sqrt(3)
Ia = cmath.rect(RATED_CURRENT, -math.acos(POWER_FACTOR))
Eaf = Va - Ia * complex(PHASE_RESISTANCE, Ls * we)
Vpk_krpm = math.sqrt(2) * ((math.sqrt(3) * abs(Eaf)) / (RATED_SPEED / 1000))

if VERBOSITY:
    print("Va :: {:.4f}".format(Va))
    print("Ia :: {:.4f}".format(Ia))
    print("Eaf :: {:.4f}".format(Eaf))
    print("Vpk_krpm :: {:.4f}".format(Vpk_krpm))

Va :: 184.7521
Ia :: 7.8210-1.1144j
Eaf :: 171.3237-43.8380j
Vpk_krpm :: 288.7840


In [241]:
Pe = 3 * (Eaf * Ia.conjugate()).real
Br = (Pe - RATE_POWER) / ws**2
tau_shaft = MOTOR_MOMENT_OF_INERTIA / Br

if VERBOSITY:
    print("Pe :: {:.4f}".format(Pe))
    print("Br :: {:.4f}m".format(Br * 1000))
    print("tau_shaft :: {:.4f}".format(tau_shaft))

Pe :: 4166.3312
Br :: 6.7411m
tau_shaft :: 1.3351


In [242]:
lambda_fm = (math.sqrt(2) * abs(Eaf)) / ((P / 2) * wr)

if VERBOSITY:
    print("lambda_fm :: {:.4f}".format(lambda_fm))

lambda_fm :: 0.7961


In [243]:
with open("../reports/" + MODEL + "_parameters.yaml", "w") as file:
    documents = yaml.dump({
            "provided_parameters": {
                "Rs": PHASE_RESISTANCE,
                "Jr": MOTOR_MOMENT_OF_INERTIA,
            }, "estimated_parameters": {
                "Ld": Ld,
                "Lq": Lq,
                "Vpk_krpm": Vpk_krpm,
                "P": P,
                "tau_shaft": tau_shaft,
            }
        }, 
        file
    )